In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from tqdm import tqdm

In [2]:
images = []
ages = []
for image in os.listdir("../input/utkface-new/crop_part1"):
    age = int(image.split("_")[0])
    ages.append(age)
    img = cv2.imread(f"../input/utkface-new/crop_part1/{image}")
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    images.append(img)

In [3]:
images = pd.Series(images,name="Images")
ages = pd.Series(ages,name="Ages")
df = pd.concat([images,ages],axis=1)
under_4 = df[df["Ages"]<=4]
under_4_new = under_4.sample(frac=0.3)
up_4 = df[df["Ages"]>4]
df = pd.concat([under_4_new,up_4],axis=0)
df = df[df["Ages"]<70]

In [4]:
X = np.array(df["Images"].tolist())
Y = np.array(df["Ages"].tolist())

In [5]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.Resize((70,70)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [6]:
class CustomDataset(Dataset):
    def __init__(self,image,label,transform):
        self.images = image
        self.labels = label
        self.transform = transform
    
    def __getitem__(self,index):
        labels = self.labels[index]
        images = self.images[index]
        images = self.transform(images)

        return images, labels
            
    def __len__(self):
        return len(self.labels)

In [7]:
dataset = CustomDataset(X,Y,transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,64,(3,3),(1,1),(1,1))
        self.conv2 = nn.Conv2d(64,128,(3,3),(1,1),(1,1))
        self.conv3 = nn.Conv2d(128,256,(3,3),(1,1),(1,1))
        
        self.fully_connect1 = nn.Linear(256*8*8,256)
        self.fully_connect2 = nn.Linear(256,128)
        self.fully_connect3 = nn.Linear(128,1)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,kernel_size=(2,2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,kernel_size=(2,2))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,kernel_size=(2,2))
        x = torch.flatten(x,start_dim=1)
        x = F.relu(self.fully_connect1(x))
        x = torch.dropout(x,0.2,train=True)
        x = F.relu(self.fully_connect2(x))
        x = torch.dropout(x,0.5,train=True)
        x = self.fully_connect3(x)
        
        return x

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model().to(device)
model.train = True

In [10]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_function = nn.MSELoss()

In [11]:
for epoch in range(20):
    train_loss = 0.0
    test_loss = 0.0
    for image, label in tqdm(train):
        image = image.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        predict = model(image)
        loss = loss_function(predict,label.float())
        loss.backward()
        optimizer.step()
        
        train_loss += loss
    
    for image, label in (test):
        image = image.to(device)
        label = label.to(device)
        
        predict = model(image)
        loss = loss_function(predict,label)
        
        test_loss += loss
    
    total_train_loss = train_loss / len(train)
    total_test_loss = test_loss / len(test)
    print(f"Epochs: {epoch+1}, Loss: {total_train_loss}, Val_Loss: {total_test_loss}")

  0%|          | 0/188 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
 99%|█████████▉| 186/188 [00:12<00:00, 34.29it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 188/188 [00:12<00:00, 14.58it/s]
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1]

Epochs: 1, Loss: 446.81573486328125, Val_Loss: 423.2130432128906


100%|██████████| 188/188 [00:05<00:00, 34.00it/s]


Epochs: 2, Loss: 409.189453125, Val_Loss: 399.49005126953125


100%|██████████| 188/188 [00:05<00:00, 33.17it/s]


...

Epochs: 18, Loss: 390.79864501953125, Val_Loss: 381.8870849609375


100%|██████████| 188/188 [00:06<00:00, 30.30it/s]


Epochs: 19, Loss: 393.3228454589844, Val_Loss: 378.16778564453125


100%|██████████| 188/188 [00:05<00:00, 33.60it/s]


Epochs: 20, Loss: 393.1153564453125, Val_Loss: 383.42706298828125


In [12]:
torch.save(model.state_dict(),"weights.pth")